Web links
Contamos con un grafo que contiene links de una página a otra. Cada vértice es una página
y cada arista un link entre ellas.
Deberá responder las siguientes preguntas usando código.
Enunciados del TP
1) ¿Cuál es el tamaño de la componente conexa más grande? ¿Cuántas componentes
conexas hay?
2) Calcular el camino mínimo de todos con todos. ¿En cuanto tiempo lo puede hacer?
¿Qué orden tiene el algoritmo? En caso de no alcanzarle el tiempo, estime cuanto
tiempo le llevaría.
3) En un grafo un triángulo es una conexión entre 3 vértices A, B y C donde:
A está conectado con B
B está conectado con C
C está conectado con A
¿Cuántos triángulos tiene el grafo?
4) Utilice el punto 2 para calcular el diámetro del grafo.
5) Google inventó un algoritmo llamado PageRank que le permitía saber qué páginas
eran más confiables según que tanto eran referenciadas. PageRank consiste en
hacer muchos random walks a lo largo del grafo y contar cuántas veces aparece
cada vértice. Los vértices que más aparecen son los de mayor PageRank. Calcule el
PageRank de los vértices del grafo.
6) La circunferencia del grafo es el largo del ciclo más largo. ¿Cuál es la circunferencia
del grafo?
Puntos extra
1) Programe una función genérica que extendiendo la definición del triángulo calcule la
cantidad de polígonos de K lados. Haga un gráfico para mostrar la cantidad de
polígonos por cantidad de lados, estimando aquellos que no pueda calcular. (+2
puntos)
2) Calcule el coeficiente de clustering del grafo (+1 punto)
3) Utilizando el punto 2, ¿cuál es el vértice con más betweenness centrality? (+2
puntos)

In [2]:
from graph import Graph

graph = Graph()

with open('web-Google.txt', 'r') as file:
    for l in file:
        if "# FromNodeId	ToNodeId" in l:
            break
    for l in file:
        if not l:
            break
        edge = tuple(int(v.replace("\n", "").replace("\t", "")) for v in l.split("\t"))
        for v in edge:
            if not graph.vertex_exists(v):
                graph.add_vertex(str(v))
        graph.add_edge(str(edge[0]), str(edge[1]))

Nodes	875713
Edges	5105039
Nodes in largest WCC	855802 (0.977)
Edges in largest WCC	5066842 (0.993)
Nodes in largest SCC	434818 (0.497)
Edges in largest SCC	3419124 (0.670)
Average clustering coefficient	0.5143
Number of triangles	13391903
Fraction of closed triangles	0.01911
Diameter (longest shortest path)	21
90-percentile effective diameter	8.1

1) ¿Cuál es el tamaño de la componente conexa más grande? ¿Cuántas componentes conexas hay? 

In [2]:

def dfs(grafo, start_vertex, visited, component):
    stack = [start_vertex]
    while stack:
        vertex = stack.pop()
        if not visited[vertex]:
            visited[vertex] = True
            component.append(vertex)
            for neighbor in grafo.get_neighbors(vertex):
                if not visited[neighbor]:
                    stack.append(neighbor)

def connected_components(grafo):
    visited = {vertex: False for vertex in grafo._graph}
    num_components = 0
    max_component_size = 0
    for vertex in grafo._graph:
        if not visited[vertex]:
            component = []
            dfs(grafo, vertex, visited, component)
            num_components += 1
            max_component_size = max(max_component_size, len(component))
    return max_component_size, num_components

max_component_size, num_components = connected_components(graph)
print("Tamaño de la componente conexa más grande:", max_component_size)
print("Número de componentes conexas:", num_components)
print("El orden del algoritmo es O(V + E). = O(875713 + 5105039) = O(5980752)")

Tamaño de la componente conexa más grande: 600493
Número de componentes conexas: 179851
El orden del algoritmo es O(V + E). = O(875713 + 5105039) = O(5980752)


2) Calcular el camino mínimo de todos con todos. ¿En cuanto tiempo lo puede hacer? ¿Qué orden tiene el algoritmo? En caso de no alcanzarle el tiempo, estime cuanto tiempo le llevaría.
4) Utilice el punto 2 para calcular el diámetro del grafo.

In [ ]:
# 2) Calcular el camino mínimo de todos con todos. ¿En cuanto tiempo lo puede hacer? ¿Qué orden tiene el algoritmo? En caso de no alcanzarle el tiempo, estime cuanto tiempo le llevaría.
# 4) Utilice el punto 2 para calcular el diámetro del grafo.

def floyd_warshall(grafo):
    vertices = list(grafo._graph.keys())
    n = len(vertices)
    dist = [[float('inf') for _ in range(n)] for _ in range(n)]
    for i in range(n):
        dist[i][i] = 0
    for vertex in vertices:
        for neighbor in grafo.get_neighbors(vertex):
            dist[vertices.index(vertex)][vertices.index(neighbor)] = grafo.get_edge_data(vertex, neighbor)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist, max(max(row) for row in dist)

dist, diameter = floyd_warshall(graph)

El diametro es 21

El orden del algoritmo es O(V^3). = O(875713^3) = O(6.715608817 x 10^17), por lo que no se puede calcular en un tiempo razonable.

3) En un grafo un triángulo es una conexión entre 3 vértices A, B y C donde: A está conectado con B, B está conectado con C, C está conectado con A ¿Cuántos triángulos tiene el grafo?

In [13]:

# 3) En un grafo un triángulo es una conexión entre 3 vértices A, B y C donde: A está conectado con B, B está conectado con C, C está conectado con A ¿Cuántos triángulos tiene el grafo?

def count_triangles(G):
    triangles = 0
    for node in G._graph.keys():
        # Obtener vecinos salientes
        neighbors = G.get_neighbors(node)
        for neighbor in neighbors:
            # Obtener vecinos salientes del vecino
            neighbors2 = G.get_neighbors(neighbor)
            for neighbor2 in neighbors2:
                neighbors3 = G.get_neighbors(neighbor2)
                if node in neighbors3:
                    triangles += 1
    return triangles
        
triangles = count_triangles(graph)
print("Cantidad de triángulos:", triangles)
print(13391903 == triangles)

Cantidad de triángulos: 11669313
False


5) Google inventó un algoritmo llamado PageRank que le permitía saber qué páginas
eran más confiables según que tanto eran referenciadas. PageRank consiste en
hacer muchos random walks a lo largo del grafo y contar cuántas veces aparece
cada vértice. Los vértices que más aparecen son los de mayor PageRank. Calcule el
PageRank de los vértices del grafo.

In [7]:
# 5) Google inventó un algoritmo llamado PageRank que le permitía saber qué páginas
# eran más confiables según que tanto eran referenciadas. PageRank consiste en
# hacer muchos random walks a lo largo del grafo y contar cuántas veces aparece
# cada vértice. Los vértices que más aparecen son los de mayor PageRank. Calcule el
# PageRank de los vértices del grafo.
def pagerank(graph, num_iterations=100, damping_factor=0.85):
    # Inicializar el PageRank de cada vértice recordar que es un grafo dirigido
    pagerank = {vertex: 1 / len(graph._graph) for vertex in graph._graph}
    num_vertices = len(graph._graph)
    for _ in range(num_iterations):
        new_pagerank = {vertex: (1 - damping_factor) / num_vertices for vertex in graph._graph}
        for vertex in graph._graph:
            neighbors = graph.get_neighbors(vertex)
            for neighbor in neighbors:
                new_pagerank[neighbor] += damping_factor * pagerank[vertex] / len(neighbors)
        pagerank = new_pagerank
    return pagerank




pagerank = pagerank(graph, 100)

n = 0
for vertex in pagerank:
    n += 1
    if n > 100:
        break
    print(f"PageRank de {vertex}: {pagerank[vertex]}")

PageRank de 0: 2.3801650064253323e-05
PageRank de 11342: 2.4645120125299453e-05
PageRank de 824020: 5.406970455209136e-06
PageRank de 867923: 2.4969271697716982e-05
PageRank de 891835: 2.4177868806261335e-05
PageRank de 27469: 4.677551695307148e-06
PageRank de 38716: 7.452091622504631e-06
PageRank de 309564: 2.193159997530595e-06
PageRank de 322178: 2.4915736386735816e-06
PageRank de 387543: 2.4915736386735816e-06
PageRank de 427436: 3.2568783380130368e-06
PageRank de 538214: 1.7297393934150114e-06
PageRank de 638706: 1.7999647097081614e-06
PageRank de 645018: 1.931528105118069e-06
PageRank de 835220: 5.059699816924727e-06
PageRank de 856657: 2.1337631505237757e-06
PageRank de 91807: 6.944422826557378e-07
PageRank de 417728: 9.936232296595615e-05
PageRank de 438493: 0.00022599630052464018
PageRank de 500627: 4.986679423992149e-06
PageRank de 535748: 4.702341602261881e-06
PageRank de 695578: 6.983565793418842e-07
PageRank de 136593: 3.1494558217399593e-06
PageRank de 414038: 2.925878732

6) La circunferencia del grafo es el largo del ciclo más largo. ¿Cuál es la circunferencia del grafo?

In [ ]:
# 6) La circunferencia del grafo es el largo del ciclo más largo. ¿Cuál es la circunferencia del grafo?

def bfs(graph, start_vertex):
    visited = {vertex: False for vertex in graph._graph}
    queue = [start_vertex]
    visited[start_vertex] = True
    while queue:
        vertex = queue.pop(0)
        for neighbor in graph.get_neighbors(vertex):
            if not visited[neighbor]:
                visited[neighbor] = True
                queue.append(neighbor)
    return sum(visited.values())

def graph_circumference(graph):
    max_circumference = 0
    for vertex in graph._graph:
        max_circumference = max(max_circumference, bfs(graph, vertex))
    return max_circumference